In [10]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# Search Expriments

In [11]:
client.search_experiments()

[<Experiment: artifact_location=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and '
  'Model Management/mlflow-scenario-examples/mlruns/2'), creation_time=1685460741764, experiment_id='2', last_update_time=1685460741764, lifecycle_stage='active', name='my-experiment', tags={}>,
 <Experiment: artifact_location=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and '
  'Model Management/mlflow-scenario-examples/mlruns/1'), creation_time=1685004904968, experiment_id='1', last_update_time=1685004904968, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685004651499, experiment_id='0', last_update_time=1685004651499, lifecycle_stage='active', name='Default', tags={}>]

# Create Experiments

In [15]:
try:
    client.create_experiment(name="my-experiment")
except Exception:
    print("experiments already exists")

experiments already exists


# Advanced Search for an Runs in an Experiment

In [14]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: c9f18d06bead4f3b8a66837015e5081b, rmse: 6.3862
run id: 0031621a93ae48ff91195a2628a37201, rmse: 6.3862
run id: 718ab510c5e844e6ae3d0d9654424f29, rmse: 6.3862
run id: eef4709501654f3a8d07c992e553b46b, rmse: 6.3862
run id: 0bc63c3bb63a4dd5b7c454f71c09f3c0, rmse: 6.3862


# Interacting with Model Registry

In this section We will use the MlflowClient instance to:

* Register a new version for the model nyc-taxi-regressor

* Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 2 was created.

* Transition the version 2 to "Staging" and adding annotations to it.

### Register a new version of the the model nyc-taxi-regressor

In [16]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [17]:
run_id = "718ab510c5e844e6ae3d0d9654424f29"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/30 23:16:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685468773581, current_stage='None', description=None, last_updated_timestamp=1685468773581, name='nyc-taxi-regressor', run_id='718ab510c5e844e6ae3d0d9654424f29', run_link=None, source=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and '
 'Model '
 'Management/mlflow-scenario-examples/mlruns/1/718ab510c5e844e6ae3d0d9654424f29/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=2>

### Get all version of a model

In [28]:
model_name = "nyc-taxi-regressor"
latest_version = client.get_latest_versions(name=model_name)

for version in latest_version:
    print(f"version: {version.version} stage: {version.current_stage}")


version: 2 stage: Staging


### Transition model stage

In [31]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685468773581, current_stage='Staging', description=('The Model Version <ModelVersion: aliases=[], '
 "creation_timestamp=1685468773581, current_stage='Staging', description=None, "
 "last_updated_timestamp=1685469399675, name='nyc-taxi-regressor', "
 "run_id='718ab510c5e844e6ae3d0d9654424f29', run_link=None, "
 "source=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment "
 "Tracking and '\n"
 " 'Model '\n"
 ' '
 "'Management/mlflow-scenario-examples/mlruns/1/718ab510c5e844e6ae3d0d9654424f29/artifacts/models_mlflow'), "
 "status='READY', status_message=None, tags={}, user_id=None, version=2> was "
 'transitioned'), last_updated_timestamp=1685469974638, name='nyc-taxi-regressor', run_id='718ab510c5e844e6ae3d0d9654424f29', run_link=None, source=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and '
 'Model '
 'Management/mlflow-scenario-examples/mlruns/1/718ab510c5e844e6ae3d0d9654424f29/artifacts/models

### Update Model Version

In [32]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The Model Version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685468773581, current_stage='Staging', description='The Model Version 2 was transitioned to Staging on 2023-05-30', last_updated_timestamp=1685469976548, name='nyc-taxi-regressor', run_id='718ab510c5e844e6ae3d0d9654424f29', run_link=None, source=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and '
 'Model '
 'Management/mlflow-scenario-examples/mlruns/1/718ab510c5e844e6ae3d0d9654424f29/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=2>

### Move version 1 to production

In [33]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685026054809, current_stage='Production', description='', last_updated_timestamp=1685470051797, name='nyc-taxi-regressor', run_id='c9f18d06bead4f3b8a66837015e5081b', run_link='', source=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and '
 'Model '
 'Management/mlflow-scenario-examples/mlruns/1/c9f18d06bead4f3b8a66837015e5081b/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

## Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

* Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
* Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
* Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
* Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
* Based on the results, update the "Production" model version accordingly.

**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [38]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [40]:
df = read_dataframe("../../data/green_tripdata_2021-03.parquet")
run_id = "c9f18d06bead4f3b8a66837015e5081b"
client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")

/var/folders/n2/3bkmj5ts56lg664b2cgfcr2m0000gn/T/ipykernel_15029/2031873194.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")


'/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and Model Management/mlflow-scenario-examples/preprocessor'

In [41]:
import pickle

with open("./preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [42]:
X_test = preprocess(df, dv)

In [43]:
target = "duration"
y_test = df[target].values

In [45]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/30 23:56:09 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[23:56:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 6.74 s, sys: 179 ms, total: 6.92 s
Wall time: 999 ms


{'rmse': 6.322777437897471}

In [46]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/30 23:56:47 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[23:56:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 6.94 s, sys: 538 ms, total: 7.47 s
Wall time: 889 ms


{'rmse': 6.322777437897471}

In [47]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1685468773581, current_stage='Production', description='The Model Version 2 was transitioned to Staging on 2023-05-30', last_updated_timestamp=1685471286957, name='nyc-taxi-regressor', run_id='718ab510c5e844e6ae3d0d9654424f29', run_link=None, source=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and '
 'Model '
 'Management/mlflow-scenario-examples/mlruns/1/718ab510c5e844e6ae3d0d9654424f29/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=2>